# Laboratorium 6 (RSA) 

Celem zajęć jest praktyczne zapoznanie się z algortmem szyfrowania asymetrycznego na przykładzie RSA.

## Przygotowanie środowska

### Materiały

* [Lista liczb pierwszych](http://compoasso.free.fr/primelistweb/page/prime/liste_online_en.php)
* [Jakaś prosta bibliotekg do GUI](https://github.com/epeios-q37/atlas-python)
* [Biblioteka do RSA](https://pycryptodome.readthedocs.io/en/latest/src/examples.html#generate-public-key-and-private-key)
* [Opis jak działa RSA](https://eduinf.waw.pl/inf/alg/001_search/0067.php#Tworzenie_kluczy_RSA)

Biblioteki

In [28]:
from typing import *
from itertools import *
import math
from pprint import pprint
import random
from collections import namedtuple
from Crypto.PublicKey import RSA

Przydatne funkcje

In [29]:
def grouper(iterable, n, fillvalue=None):
    """Collect data into fixed-length chunks or blocks"""
    # grouper('ABCDEFG', 3, 'x') --> ABC DEF Gxx"
    args = [iter(iterable)] * n
    return zip_longest(*args, fillvalue=fillvalue)


list(grouper(range(10), 3, 0))

[(0, 1, 2), (3, 4, 5), (6, 7, 8), (9, 0, 0)]

In [30]:
def join_numbers(numbers : Iterable[int]) -> int:
    """Bijection that maps list of numbers to one big number"""
    return int(''.join(str(n + 100) for n in numbers))
    
join_numbers([2, 17, 0, 5])

102117100105

In [31]:
def separate_numbers(number : int) -> Iterable[int]:
    """Bijection that one big number to maps list of numbers"""
    number = str(number)
    return (int(''.join(n)) - 100 for n in grouper(number, 3))
    
list(separate_numbers(join_numbers([2, 17, 0, 5])))

[2, 17, 0, 5]

## Zadania z iSoda

### Zadnie 1

Napisz funkcje, która będzie przekształcać tekst w **wektor liczb** o określonej długości. Określ ilość znaków, którą chcesz kodować jednocześnie.

In [32]:
def text_to_numbers(text : str, block_size = 10) -> List[int]:
    text = text.encode()
    fill_value = 0
    
    return (join_numbers(block) for block in grouper(text, block_size, fill_value))

text = 'Moja całkiem długa wiadomość deszcz pada deszcz pdada deszcz pada jesienny'

list(text_to_numbers(text))

[177211206197132199197297230207,
 205201209132200297230217203197,
 132219205197200211209211297255,
 296235132200201215222199222132,
 212197200197132200201215222199,
 222132212200197200197132200201,
 215222199222132212197200197132,
 206201215205201210210221100100]

### Zadnie 2

Napisz funkcje odwrotną, czyli przekształcającą wektor liczb w tekst.

In [33]:
def numbers_to_text(numbers : Iterable[int]) -> str:
    fill_value = 0
    
    data = chain.from_iterable(map(separate_numbers, numbers))
    data = takewhile(lambda n: n != fill_value, data) # Trim padding
    
    return bytes(data).decode()

number_vector = text_to_numbers(text, 20)

numbers_to_text(number_vector)

'Moja całkiem długa wiadomość deszcz pada deszcz pdada deszcz pada jesienny'

### Zadanie 3

Zaimplementuj prosty test pierwszości dla liczby (np. sito Erastotenesa, test Fermata).

In [34]:
# Primes
2063
10691
274090011823

test_prime = 10691
big_test_prime = 274090011823

#### Test Fermata

##### Teoria

Małe twierdzenie Fermata mówi, że jeśli **p** jest liczbą pierwszą i **a** nie dzieli się przez **p**, to  

\begin{equation}
a^{p-1} \equiv{} 1 \ \mathrm{mod}\ p
\end{equation}

Aby stwierdzić, czy **p** jest pierwsza, można wybrać kilka losowych wartości a względnie pierwszych z **p** i sprawdzić, czy ta równość jest dla nich spełniona. Jeśli dla którejkolwiek z nich nie jest, to na pewno **p** jest liczbą złożoną. Jeśli wszystkie ją spełniają, **p** jest prawdopodobnie liczbą pierwszą albo pseudopierwszą

##### Słabe

In [38]:
# With ** operator

def fermat_test_bad(number : int, tries = 100):
    assert(tries > 0)
    assert(number > 2)
    
    for _ in range(tries):
        a = random.randint(2, number - 1)
        if a ** (number-1) % number != 1:
            return False
        
    return True

fermat_test_bad(test_prime)

True

In [36]:
%%timeit
fermat_test_bad(test_prime)

117 ms ± 1.57 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [84]:
%%timeit
pass
# fermat_test_bad(big_test_prime)

6.1 ns ± 0.24 ns per loop (mean ± std. dev. of 7 runs, 100000000 loops each)


##### Ok

In [41]:
# With buildin.pow

def fermat_test(number : int, tries = 100) -> bool:
    """Iterative Fermat test"""
    assert(tries > 0)
    if number % 2 == 0: return False
    assert(number > 2)
    
    p = number
    for _ in range(tries):
        a = random.randint(2, number - 1)
        if pow(a, p-1, p) != 1:
            return False
        
    return True

fermat_test(test_prime)
# print(fermat_test.__doc__)

True

In [42]:
%%timeit
fermat_test(test_prime)

185 µs ± 9.57 µs per loop (mean ± std. dev. of 7 runs, 10000 loops each)


In [43]:
%%timeit
fermat_test(big_test_prime)

787 µs ± 15.9 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


In [46]:
# Functional

def fermat_test(number : int, tries = 100):
    """Functional Fermat test"""
    assert(tries > 0)
    if number == 2: return True
    if number % 2 == 0: return False
    assert(number > 2)
    
    p = number
    stuff = (pow(random.randint(2, number - 1), p-1, p) != 1 for _ in range(tries))
    
    result = list(takewhile(lambda x: x == False, stuff))
    return len(result) == tries

# fermat_test(2 ** 40 - 1)
print(fermat_test.__doc__)
fermat_test(test_prime - 1)

Functional Fermat test


False

In [45]:
%%timeit
fermat_test(test_prime)

204 µs ± 4.82 µs per loop (mean ± std. dev. of 7 runs, 10000 loops each)


In [47]:
%%timeit
fermat_test(big_test_prime)

805 µs ± 27.6 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


#### Sprawdzanie dzielników

In [48]:
def naive_primity_test(number : int) -> bool:
    if number <= 1: return False
    if number == 2: return True
    if number % 2 == 0: return False
    
    for i in range(3, math.floor(math.sqrt(number)) + 1, 2):
        if number % i == 0:
            return False
        
    return True

naive_primity_test(21)

False

In [49]:
%%timeit
naive_primity_test(test_prime)

2.22 µs ± 160 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)


In [50]:
%%timeit
naive_primity_test(big_test_prime)

21.4 ms ± 1.35 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


### Zadanie 4

Zaimplementuj algorytm Euklidesa do sprawdzenia największego wspólnego dzielnika.

In [51]:
def greatest_common_divisor(a : int, b : int) -> int:
    """GCD using Euclidean algorythm"""
    if a < b: a, b = b, a
    
    while b:
        a, b = b, a%b
    return a

In [52]:
print(greatest_common_divisor(282, 78))
print(greatest_common_divisor(78, 282))

6
6


### Zadanie 5

Napisz funkcję wykorzystającą algorytm Euklidesa do sprawdzenia czy liczby są względnie pierwsze.

In [53]:
def are_semi_prime(a : int, b : int) -> bool:
    return greatest_common_divisor(a, b) == 1

In [54]:
print(are_semi_prime(14, 7))
print(are_semi_prime(21, 10))

False
True


### Zadanie 6

Napisz funkcję wykorzystającą algorytm Euklidesa do obliczenia liczby odwrotnej modulo.

In [55]:
def egcd(a : int, b : int) -> Tuple[int, int, int]:
    if a == 0:
        return (b, 0, 1)
    else:
        g, y, x = egcd(b % a, a)
        return (g, x - (b // a) * y, y)
    
def modinv(number : int, modulo : int) -> int:
    g, x, y = egcd(number, modulo)
    
    if g != 1:
        raise Exception('modular inverse does not exist')
    else:
        return x % modulo
    
modinv(17, 43)

38

### Zadanie 7

Zaimplementuj algorytm szybkiego potęgowania w artmetyce modularnej.

In [57]:
def power(number : int, power: int, modulo : int) -> int: 
    number %= modulo
    result = 1
  
    while (power): 
        if ((power & 1) == 1): # Quicly chceck if it's odd
            result = (result * number) % modulo
            power -= 1
        else:
            # Speedup trick
            power //= 2 
            number = (number ** 2) % modulo
          
    return result

print(power(2, 5, 13))
print(pow(2, 5, 13))

6
6


### Zadanie 8

Napisz pełny program generujący klucze RSA.

In [59]:
# TODO: Prime generation - maby load from file?
def get_primes() -> Tuple[int, int]:
    """Returns pair of prime numbers for RSA kyes"""
    p = 409
    q = 1747
    
    return p, q

def find_semi_prime(number : int) -> int:
    """Return r such that GCD(number, r) == 1 and r % 2 == 1"""
    tmp = number - (not (number & 1)) - 2
    for i in range(tmp, 2, -2):
        if are_semi_prime(i, number):
            return i
        
    raise ValueError(f'Unable to find semiprime to {number}')

In [60]:
Key = namedtuple('Key', ['exponent', 'mod'])

def create_RSA_keys() -> NamedTuple:
    p, q = get_primes()
    fi = (p - 1) * (q - 1)
    n = p * q
    e = find_semi_prime(n)
    d = modinv(e, fi)
    
    public_key = Key(e, n)
    private_key = Key(d, n)
    
    return namedtuple('RSAKeyPair', ['public_key', 'private_key'])(public_key, private_key)

create_RSA_keys()

RSAKeyPair(public_key=Key(exponent=714521, mod=714523), private_key=Key(exponent=409289, mod=714523))

### Zadanie 9

Napis program realizujący szyfrowanie RSA

Testowanie działania

In [63]:
keys = create_RSA_keys()
print(keys)

to_encrypt = 123
print('To encrypt:', to_encrypt)

encrypted = pow(to_encrypt, *keys.public_key)
print('Encrypted:', encrypted)

decrypted = pow(encrypted, *keys.private_key)
print('Decrypted:', decrypted)

RSAKeyPair(public_key=Key(exponent=714521, mod=714523), private_key=Key(exponent=409289, mod=714523))
To encrypt: 123
Encrypted: 386934
Decrypted: 123


In [68]:
def RSA_encrypt(text : str, key : Key) -> Iterable[int]:
    encoded = text_to_numbers(text, 2) # For bigger block size take bigger primes
    return (pow(block, *key) for block in encoded)

def RSA_decrypt(data : Iterable[int], key : Key) -> str:
    decrypted = (pow(block, *key) for block in data)
    return numbers_to_text(decrypted)

In [69]:
text = 'Moja całkiem długa wiadomość deszcz pada deszcz pdada deszcz pada jesienny'
keys = create_RSA_keys()

encrypted = list(RSA_encrypt(text, keys.public_key))
print(encrypted)

decrypted = RSA_decrypt(encrypted, keys.private_key)
print(decrypted)

[59655, 568406, 291408, 63232, 69087, 609593, 8773, 64661, 505083, 430247, 75859, 22769, 538402, 198288, 316447, 126644, 264709, 664709, 409603, 298472, 386228, 101749, 264709, 664709, 409603, 298472, 582925, 510748, 122959, 263275, 493373, 437331, 469752, 510748, 122959, 622837, 283337, 433269, 662102]
Moja całkiem długa wiadomość deszcz pada deszcz pdada deszcz pada jesienny


### Zadanie 10

Napisz program korzystający z PyCrypto do szyfrowania plików. Zaproponuj metodę ochrony klucza prywatnego.

Do ochrony klucza prywatnego można wykożystać kryptografię symetryczną

In [90]:
key = RSA.generate(2048)
# pprint(dir(key))
private_key = key.exportKey()
file_out = open("private.pem", "wb")
file_out.write(private_key)

public_key = key.publickey().exportKey()
file_out = open("public.pem", "wb")
file_out.write(public_key)

data = None
file_to_encrypt = 'file.txt'
with open(file_to_encrypt) as file:
    data = file.read()
    
print('Dane do zaszyfrowania:')
print(data)
print()

# pprint(list(text_to_numbers(data)))

encrypted = [key.publickey().encrypt(block, None)[0] for block in text_to_numbers(data)]
print('Fragment szyfrogramu:')
pprint(encrypted[:2])
print()

recovered = numbers_to_text((key.decrypt(block) for block in encrypted))
print('Odszyfrowane dane:')
print(recovered)
print()

# tmp = key.publickey().encrypt(165169183132140165200218197210, 2)[0]
# help(key.publickey().encrypt)

# key.decrypt(tmp)

Dane do zaszyfrowania:
AES (Advanced Encryption Standard) is a symmetric block cipher standardized by NIST . It has a fixed data block size of 16 bytes. Its keys can be 128, 192, or 256 bits long.

AES is very fast and secure, and it is the de facto standard for symmetric encryption.

As an example, encryption can be done as follows:

Fragment szyfrogramu:
[18306445146818202277665770039620598066360008518341468223305693688972437052089785867877342394267044375918146780807824483542386918637592687147948635775954445827806316611334992989252369334620111584805057269801205369338053489649400247439667557812426939979102127247177909382337426519527252182373601624471141997662956791512526078488719365758547039990844212678324811540296148471735292986104496349897812286831502315498594123860536581323559269707927842188137768991960354109379599150494965938568178235850315781236113897116628297443189485953833042800670238156246952225425833934439370988788818894062420924071662661752069380682988,
 10976047301762390573

In [72]:
help(key.publickey().encrypt)

Help on method encrypt in module Crypto.PublicKey.RSA:

encrypt(plaintext, K) method of Crypto.PublicKey.RSA._RSAobj instance
    Encrypt a piece of data with RSA.
    
    :Parameter plaintext: The piece of data to encrypt with RSA. It may not
     be numerically larger than the RSA module (**n**).
    :Type plaintext: byte string or long
    
    :Parameter K: A random parameter (*for compatibility only. This
     value will be ignored*)
    :Type K: byte string or long
    
    :attention: this function performs the plain, primitive RSA encryption
     (*textbook*). In real applications, you always need to use proper
     cryptographic padding, and you should not directly encrypt data with
     this method. Failure to do so may lead to security vulnerabilities.
     It is recommended to use modules
     `Crypto.Cipher.PKCS1_OAEP` or `Crypto.Cipher.PKCS1_v1_5` instead.
    
    :Return: A tuple with two items. The first item is the ciphertext
     of the same type as the plaintext (s